In [1]:
import pandas as pd

### Import from Qlik

In [2]:
Total_TE_2 = pd.read_csv("X:/QSDOCS/PRD/QVD/Final/Total_TE_2.csv")

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\3800962447.py:1: DtypeWarning: Columns (1,4,7,10,11,12,14,16,17,18,19,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  Total_TE_2 = pd.read_csv("X:/QSDOCS/PRD/QVD/Final/Total_TE_2.csv")


### Cleansing Start

In [3]:
# Standardize the YOUY and CAGR values
mode_yoy = Total_TE_2.groupby("PROGRAMME_NAME")["YOY_23_24"].apply(lambda x: x.mode().iat[0] if not x.mode().empty else "0%")
mode_cagr = Total_TE_2.groupby("PROGRAMME_NAME")["CAGR_25_29"].apply(lambda x: x.mode().iat[0] if not x.mode().empty else "0%")

In [4]:
#To standardise the CAGR and YOY values
Total_TE_2["YOY_23_24"] = Total_TE_2["PROGRAMME_NAME"].map(mode_yoy)
Total_TE_2["CAGR_25_29"] = Total_TE_2["PROGRAMME_NAME"].map(mode_cagr)

In [5]:
#Replace the Not Mapped values with 0%
Total_TE_2["YOY_23_24"] = Total_TE_2["YOY_23_24"].replace("Not Mapped", "0%")
Total_TE_2["CAGR_25_29"] = Total_TE_2["CAGR_25_29"].replace("Not Mapped", "0%")

In [6]:
Total_TE_2['YOY_23_24'] = (
    Total_TE_2['YOY_23_24']
    .replace('Not Mapped', '0.00%')
    .astype(str)
    .str.rstrip('%')
    .pipe(lambda s: pd.to_numeric(s, errors='coerce'))
    .fillna(0.00)
    .div(100)  # Use pandas' div() instead of / operator
    .round(4)  # Now this will work on the Series
    .mul(100)  # Multiply back by 100 if needed
    .astype(str)
    + '%'      # Add back the % sign
)

In [7]:
# For 'CAGR_25_29' Column
Total_TE_2['CAGR_25_29'] = (
    Total_TE_2['CAGR_25_29']
    .replace('Not Mapped', '0.00%')
    .astype(str)
    .str.rstrip('%')
    .pipe(lambda s: pd.to_numeric(s, errors='coerce'))
    .fillna(0.00)
    .div(100)  # Use pandas' div() instead of / operator
    .round(4)  # Now this will work on the Series
    .mul(100)  # Multiply back by 100 if needed
    .astype(str)
    + '%'      # Add back the % sign
)

In [8]:
#To seperate Taylors University and Taylors College
Taylors_df = Total_TE_2[Total_TE_2['IPTS'].isin(["Taylor's University", "Taylor's College"])]

In [9]:
Taylors_df = Taylors_df[['YEAR','PROGRAMME_NAME','IPTS','YOY_23_24','CAGR_25_29','TE','FLAG_COMPETITOR','Fact_Type']]
Taylors_df.loc[(Taylors_df['YEAR'] == 2024), 'YEAR'] = '2024F'
Taylors_df.loc[(Taylors_df['YEAR'] == 2025), 'YEAR'] = '2025F'
Taylors_df

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\2183376003.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024F' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Taylors_df.loc[(Taylors_df['YEAR'] == 2024), 'YEAR'] = '2024F'


,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
27520,2025F,NaN,Taylor's College,0.0%,0.0%,0.000000,Y,Actual
27521,2025F,NaN,Taylor's College,0.0%,0.0%,0.000000,Y,Actual
27522,2025F,NaN,Taylor's College,0.0%,0.0%,0.000000,Y,Actual
27523,2025F,NaN,Taylor's College,0.0%,0.0%,0.000000,Y,Actual
27524,2025F,NaN,Taylor's College,0.0%,0.0%,0.000000,Y,Actual
...,...,...,...,...,...,...,...,...
28560,2023,POSTGRADUATE CERTIFICATE IN EDUCATION,Taylor's University,0.0%,0.0%,32.083333,Y,Actual
28561,2024F,POSTGRADUATE CERTIFICATE IN EDUCATION,Taylor's University,0.0%,0.0%,42.416667,Y,Actual
28562,2025F,POSTGRADUATE CERTIFICATE IN EDUCATION,Taylor's University,0.0%,0.0%,NaN,Y,Actual
28563,2025F,SOUTH AUSTRALIAN CERTIFICATE OF EDUCATION,Taylor's University,4.16%,3.44%,0.000000,Y,Actual


In [10]:
#Standardize the TE values
Taylors_df['TE'] = pd.to_numeric(Taylors_df['TE'], errors='coerce').fillna(0).round().astype(int)

In [11]:
#Bring out only 2024F data and keep the only 2018 - 2023 data
Taylors_df_2024F = Taylors_df[Taylors_df['YEAR'] == '2024F']
Taylors_df_2023F = Taylors_df[~Taylors_df['YEAR'].isin(['2024F', '2025F'])]

In [12]:
Taylors_df_2024F['CAGR_25_29'] = Taylors_df_2024F['CAGR_25_29'].str.rstrip('%').astype(float)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\1081873030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Taylors_df_2024F['CAGR_25_29'] = Taylors_df_2024F['CAGR_25_29'].str.rstrip('%').astype(float)


In [13]:
Taylors_df_2024F

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
27556,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,42,Y,Actual
27560,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,48,Y,Actual
27565,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,11,Y,Actual
27570,2024F,ACCA FOUNDATION IN ACCOUNTANCY,Taylor's College,5.99%,2.74,14,Y,Actual
27639,2024F,CAMBRIDGE A-LEVEL,Taylor's College,-2.9899999999999998%,-2.66,860,Y,Actual
...,...,...,...,...,...,...,...,...
28543,2024F,MASTER OF SCIENCE IN MEDICAL SCIENCE,Taylor's University,5.0%,5.00,7,Y,Actual
28546,2024F,MASTER OF SCIENCE IN VIRTUAL DESIGN AND CONSTR...,Taylor's University,5.0%,5.00,22,Y,Actual
28549,2024F,MASTER OF TEACHING AND LEARNING (ODL),Taylor's University,18.05%,9.02,41,Y,Actual
28558,2024F,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.00,13,Y,Actual


In [14]:
Taylors_df_2024F['2025F'] = Taylors_df_2024F['TE'] * (1 + Taylors_df_2024F['CAGR_25_29'] / 100) 

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\2896518341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Taylors_df_2024F['2025F'] = Taylors_df_2024F['TE'] * (1 + Taylors_df_2024F['CAGR_25_29'] / 100)


In [15]:
Taylors_df_2024F

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type,2025F
27556,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,42,Y,Actual,41.8194
27560,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,48,Y,Actual,47.7936
27565,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,11,Y,Actual,10.9527
27570,2024F,ACCA FOUNDATION IN ACCOUNTANCY,Taylor's College,5.99%,2.74,14,Y,Actual,14.3836
27639,2024F,CAMBRIDGE A-LEVEL,Taylor's College,-2.9899999999999998%,-2.66,860,Y,Actual,837.1240
...,...,...,...,...,...,...,...,...,...
28543,2024F,MASTER OF SCIENCE IN MEDICAL SCIENCE,Taylor's University,5.0%,5.00,7,Y,Actual,7.3500
28546,2024F,MASTER OF SCIENCE IN VIRTUAL DESIGN AND CONSTR...,Taylor's University,5.0%,5.00,22,Y,Actual,23.1000
28549,2024F,MASTER OF TEACHING AND LEARNING (ODL),Taylor's University,18.05%,9.02,41,Y,Actual,44.6982
28558,2024F,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.00,13,Y,Actual,13.0000


In [16]:
Taylors_df_2024F['2025F'] = pd.to_numeric(Taylors_df_2024F['2025F'], errors='coerce').round().astype(int)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\3532321463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Taylors_df_2024F['2025F'] = pd.to_numeric(Taylors_df_2024F['2025F'], errors='coerce').round().astype(int)


In [17]:
df_2025F = Taylors_df_2024F.copy()
df_2025F['YEAR'] = '2025F'
df_2025F['TE'] = df_2025F['2025F']

In [18]:
df_2025F = df_2025F.drop(columns=['2025F'])

In [19]:
Taylors_df_2024F = Taylors_df_2024F.drop(columns=['2025F'])

In [20]:
Taylors_df_2024F = pd.concat([Taylors_df_2024F, df_2025F], ignore_index=True)

In [21]:
Taylors_df_2024F

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
0,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,42,Y,Actual
1,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,48,Y,Actual
2,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43,11,Y,Actual
3,2024F,ACCA FOUNDATION IN ACCOUNTANCY,Taylor's College,5.99%,2.74,14,Y,Actual
4,2024F,CAMBRIDGE A-LEVEL,Taylor's College,-2.9899999999999998%,-2.66,860,Y,Actual
...,...,...,...,...,...,...,...,...
231,2025F,MASTER OF SCIENCE IN MEDICAL SCIENCE,Taylor's University,5.0%,5.00,7,Y,Actual
232,2025F,MASTER OF SCIENCE IN VIRTUAL DESIGN AND CONSTR...,Taylor's University,5.0%,5.00,23,Y,Actual
233,2025F,MASTER OF TEACHING AND LEARNING (ODL),Taylor's University,18.05%,9.02,45,Y,Actual
234,2025F,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.00,13,Y,Actual


In [22]:
Taylors_df_2024F['CAGR_25_29'] = Taylors_df_2024F['CAGR_25_29'].astype(str) + '%'

In [23]:
Taylors_df_2023F

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
27528,2018,NaN,Taylor's College,0.0%,0.0%,27,Y,Actual
27537,2018,NaN,Taylor's College,0.0%,0.0%,12,Y,Actual
27551,2019,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,22,Y,Actual
27552,2020,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,88,Y,Actual
27553,2021,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,94,Y,Actual
...,...,...,...,...,...,...,...,...
28554,2020,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,37,Y,Actual
28555,2021,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,39,Y,Actual
28556,2022,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,29,Y,Actual
28557,2023,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,25,Y,Actual


In [24]:
Taylors_df = pd.concat([Taylors_df_2024F, Taylors_df_2023F], ignore_index=True)

In [25]:
Taylors_df

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
0,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,42,Y,Actual
1,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,48,Y,Actual
2,2024F,ACCA (FT/PT),Taylor's College,0.52%,-0.43%,11,Y,Actual
3,2024F,ACCA FOUNDATION IN ACCOUNTANCY,Taylor's College,5.99%,2.74%,14,Y,Actual
4,2024F,CAMBRIDGE A-LEVEL,Taylor's College,-2.9899999999999998%,-2.6599999999999997%,860,Y,Actual
...,...,...,...,...,...,...,...,...
852,2020,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,37,Y,Actual
853,2021,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,39,Y,Actual
854,2022,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,29,Y,Actual
855,2023,POST-GRADUATE CERTIFICATE IN TEACHING AND LEAR...,Taylor's University,0.0%,0.0%,25,Y,Actual


In [ ]:

Key_Competitor_Y = Total_TE_2[
    (Total_TE_2['FLAG_COMPETITOR'] == 'Y') & 
    (~Total_TE_2['IPTS'].isin(["Taylor's University", "Taylor's College"]))
]

In [27]:
competitor_2024_2025 = Key_Competitor_Y[
    (Key_Competitor_Y['YEAR'] >= 2024) & 
    (Key_Competitor_Y['YEAR'] <= 2025)
]

In [28]:
competitor_2024_2025 = competitor_2024_2025[['YEAR','PROGRAMME_NAME','IPTS','YOY_23_24','CAGR_25_29','TE','FLAG_COMPETITOR','Fact_Type']]

In [29]:
competitor_2024_Y = competitor_2024_2025[competitor_2024_2025['YEAR'] == 2024]

In [31]:
competitor_2024_Y['CAGR_25_29'] = competitor_2024_Y['CAGR_25_29'].str.rstrip('%').astype(float)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\2915836485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_2024_Y['CAGR_25_29'] = competitor_2024_Y['CAGR_25_29'].str.rstrip('%').astype(float)


In [32]:
competitor_2024_Y['2025F'] = competitor_2024_Y['TE'] * (1 + competitor_2024_Y['CAGR_25_29'] / 100) 

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\3612837860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_2024_Y['2025F'] = competitor_2024_Y['TE'] * (1 + competitor_2024_Y['CAGR_25_29'] / 100)


In [33]:
competitor_2024_Y['2025F'] = pd.to_numeric(competitor_2024_Y['2025F'], errors='coerce').fillna(0).round().astype(int)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\4051998688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_2024_Y['2025F'] = pd.to_numeric(competitor_2024_Y['2025F'], errors='coerce').fillna(0).round().astype(int)


In [35]:
competitor_2025 = competitor_2024_Y.copy()
competitor_2025['YEAR'] = '2025F'
competitor_2025['TE'] = competitor_2025['2025F']

competitor_2025 = competitor_2025.drop(columns=['2025F'])
competitor_2024_Y = competitor_2024_Y.drop(columns=['2025F'])

competitor_2024_2025 = pd.concat([competitor_2024_Y, competitor_2025], ignore_index=True)

In [36]:
competitor_2024_2025

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
0,2024,"BACHELOR OF MEDICINE, BACHELOR OF SURGERY - MBBS",AIMST University,0.11%,3.00,998.0967,Y,Actual
1,2024,MASTER OF TEACHING AND LEARNING (ODL),Asia e University,18.05%,9.02,996.3420,Y,Actual
2,2024,BACHELOR OF ARTS (HONOURS) ACCOUNTING AND FINANCE,Asia Pacific University of Technology and Inno...,-5.0%,0.11,288.8000,Y,Actual
3,2024,BACHELOR OF BUSINESS (HONOURS),Asia Pacific University of Technology and Inno...,1.9900000000000002%,2.00,407.8800,Y,Actual
4,2024,BACHELOR OF BUSINESS (HONOURS) (INTERNATIONAL ...,Asia Pacific University of Technology and Inno...,1.9900000000000002%,1.00,260.0745,Y,Actual
...,...,...,...,...,...,...,...,...
6787,2025F,DIPLOMA IN CULINARY ARTS,OTHERS,10.0%,5.92,0.0000,Y,Actual
6788,2025F,DIPLOMA IN HOTEL MANAGEMENT,OTHERS,20.0%,5.33,0.0000,Y,Actual
6789,2025F,FOUNDATION IN ARTS,OTHERS,3.0%,2.37,0.0000,Y,Actual
6790,2025F,DIPLOMA IN CULINARY ARTS,OTHERS,10.0%,5.92,0.0000,Y,Actual


In [37]:
competitor_2024_2025['YEAR'] = competitor_2024_2025['YEAR'].replace(2024, '2024F')

In [38]:
competitor_2024_2025['CAGR_25_29'] = competitor_2024_2025['CAGR_25_29'].astype(str) + '%'

In [39]:
competitor_before_2024 = Key_Competitor_Y[
    Key_Competitor_Y['YEAR'] < 2024
]

In [40]:
competitor_before_2024 = competitor_before_2024[['YEAR','PROGRAMME_NAME','IPTS','YOY_23_24','CAGR_25_29','TE','FLAG_COMPETITOR','Fact_Type']]

In [41]:
competitor_before_2024['TE'] = pd.to_numeric(competitor_before_2024['TE'], errors='coerce').fillna(0).round().astype(int)

In [43]:
Key_Competitor_Y = pd.concat([competitor_before_2024, competitor_2024_2025], axis=0)

In [45]:
Key_Competitor_Y = Key_Competitor_Y.sort_values(by=['PROGRAMME_NAME', 'YEAR'])

In [46]:
Key_Competitor_N = Total_TE_2[Total_TE_2['FLAG_COMPETITOR'] == 'N']

In [48]:
Key_Competitor_N = Key_Competitor_N[['YEAR','PROGRAMME_NAME','IPTS','YOY_23_24','CAGR_25_29','TE','FLAG_COMPETITOR','Fact_Type']]

In [49]:
competitor_before_2024_N = Key_Competitor_N[
    Key_Competitor_N['YEAR'] < 2024
]

In [51]:
competitor_before_2024_N['TE'] = pd.to_numeric(competitor_before_2024_N['TE'], errors='coerce').fillna(0).round().astype(int)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\1059102716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_before_2024_N['TE'] = pd.to_numeric(competitor_before_2024_N['TE'], errors='coerce').fillna(0).round().astype(int)


In [52]:
competitor_before_2024_N['YOY_23_24'] = competitor_before_2024_N['YOY_23_24'].str.rstrip('%').astype(float)
competitor_before_2024_N['CAGR_25_29'] = competitor_before_2024_N['CAGR_25_29'].str.rstrip('%').astype(float)

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\624725160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_before_2024_N['YOY_23_24'] = competitor_before_2024_N['YOY_23_24'].str.rstrip('%').astype(float)
C:\Users\112363\AppData\Local\Temp\ipykernel_4568\624725160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_before_2024_N['CAGR_25_29'] = competitor_before_2024_N['CAGR_25_29'].str.rstrip('%').astype(float)


In [53]:
competitor_2023N = competitor_before_2024_N[competitor_before_2024_N['YEAR'] == 2023]
competitor_not_2023N = competitor_before_2024_N[competitor_before_2024_N['YEAR'] != 2023]

In [55]:
competitor_2023N['IPTS']="OTHERS"

C:\Users\112363\AppData\Local\Temp\ipykernel_4568\2379431372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitor_2023N['IPTS']="OTHERS"


In [56]:
competitor_2023N_v1 = competitor_2023N.groupby(['YEAR','PROGRAMME_NAME','IPTS','YOY_23_24','CAGR_25_29','FLAG_COMPETITOR','Fact_Type'])['TE'].sum().reset_index()

In [58]:
competitor_2023N_v1['2024F'] = competitor_2023N_v1['TE'] * (1 + competitor_2023N_v1['YOY_23_24'] / 100) 

In [59]:
competitor_2023N_v1['2025F'] = competitor_2023N_v1['TE'] * (1 + competitor_2023N_v1['YOY_23_24'] / 100) * (1 + competitor_2023N_v1['CAGR_25_29'] / 100) 

In [85]:
competitor_2024N

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
176,2024F,Not Indicated,OTHERS,0.00,0.00,55,N,Actual
177,2024F,Not Indicated,OTHERS,0.00,0.00,93,N,Actual
188,2024F,Not Indicated,OTHERS,0.00,0.00,288,N,Actual
196,2024F,BACHELOR OF ARTS (HONOURS) ACCOUNTING AND FINANCE,OTHERS,-5.00,0.11,66,N,Actual
200,2024F,BACHELOR OF BIOMEDICAL SCIENCE (HONOURS),OTHERS,1.05,1.05,60,N,Actual
...,...,...,...,...,...,...,...,...
80631,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80632,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80633,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80647,2024F,DIPLOMA IN EVENT MANAGEMENT,OTHERS,10.00,4.63,0,N,Actual


In [ ]:
competitor_2024N = competitor_2023N.copy()
competitor_2024N['YEAR'] = '2024F'
#competitor_2024N['TE'] = competitor_2024N['2024F']
#competitor_2024N = competitor_2024N.drop(columns=['2025F','2025'])
competitor_2024N 

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
176,2024F,Not Indicated,OTHERS,0.00,0.00,55,N,Actual
177,2024F,Not Indicated,OTHERS,0.00,0.00,93,N,Actual
188,2024F,Not Indicated,OTHERS,0.00,0.00,288,N,Actual
196,2024F,BACHELOR OF ARTS (HONOURS) ACCOUNTING AND FINANCE,OTHERS,-5.00,0.11,66,N,Actual
200,2024F,BACHELOR OF BIOMEDICAL SCIENCE (HONOURS),OTHERS,1.05,1.05,60,N,Actual
...,...,...,...,...,...,...,...,...
80631,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80632,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80633,2024F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80647,2024F,DIPLOMA IN EVENT MANAGEMENT,OTHERS,10.00,4.63,0,N,Actual


In [89]:
#competitor_2024N = competitor_2024N.drop(columns=['2024F'])

In [ ]:
competitor_2025N = competitor_2023N.copy()
competitor_2025N['YEAR'] = '2025F'
#competitor_2025N['TE'] = competitor_2025N['2025F']
#competitor_2025N = competitor_2025N.drop(columns=['2024F','2025'])
competitor_2025N 

,YEAR,PROGRAMME_NAME,IPTS,YOY_23_24,CAGR_25_29,TE,FLAG_COMPETITOR,Fact_Type
176,2025F,Not Indicated,OTHERS,0.00,0.00,55,N,Actual
177,2025F,Not Indicated,OTHERS,0.00,0.00,93,N,Actual
188,2025F,Not Indicated,OTHERS,0.00,0.00,288,N,Actual
196,2025F,BACHELOR OF ARTS (HONOURS) ACCOUNTING AND FINANCE,OTHERS,-5.00,0.11,66,N,Actual
200,2025F,BACHELOR OF BIOMEDICAL SCIENCE (HONOURS),OTHERS,1.05,1.05,60,N,Actual
...,...,...,...,...,...,...,...,...
80631,2025F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80632,2025F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80633,2025F,Not Indicated,OTHERS,0.00,0.00,0,N,Actual
80647,2025F,DIPLOMA IN EVENT MANAGEMENT,OTHERS,10.00,4.63,0,N,Actual


In [93]:
#competitor_2025N = competitor_2025N.drop(columns=['2025F'])

In [69]:
not_competitor = pd.concat([competitor_2024N, competitor_not_2023N,competitor_2023N,competitor_2025N], ignore_index=True)

In [71]:
#Standardize the TE values
not_competitor['TE'] = pd.to_numeric(not_competitor['TE'], errors='coerce').fillna(0).round().astype(int)

In [72]:
not_competitor['YOY_23_24'] = not_competitor['YOY_23_24'].astype(str) + '%'
not_competitor['CAGR_25_29'] = not_competitor['CAGR_25_29'].astype(str) + '%'

In [73]:
Key_Competitor_N=not_competitor

In [75]:
Key_Competitor_N['IPTS'] = 'OTHERS'

In [77]:
Key_Competitor_Y_v1 = Key_Competitor_Y[Key_Competitor_Y['IPTS']!= "OTHERS"]

In [80]:
Key_Competitor_All = pd.concat([Taylors_df, Key_Competitor_Y_v1,Key_Competitor_N], ignore_index=True)

In [82]:
Key_Competitor_All_v1 = Key_Competitor_All[Key_Competitor_All['PROGRAMME_NAME'] != "Not Indicated"]

In [ ]:
Key_Competitor_All_v1.to_excel("C:/Users/112363/OneDrive - Taylor's Education Group/DWH_WIP/PEMO/Key_Competitor.xlsx", sheet_name="CPM_2025", index=False)